In [ ]:
!pip install -q -U pypdf faiss-cpu langchain langchain-community huggingface_hub sentence-transformers langchain-google-genai tavily-python langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma


In [ ]:
loader = PyPDFDirectoryLoader("/content/data")
data = loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoModel
model = AutoModel.from_pretrained('mixedbread-ai/mxbai-embed-large-v1', trust_remote_code=True)

# model_name = "jinaai/jina-embeddings-v2-base-en"
model_name = "mixedbread-ai/mxbai-embed-large-v1"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
)


In [ ]:
loader = PyPDFDirectoryLoader("/content/data")
data = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(data, embeddings)
retriever = db.as_retriever()


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
text_llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key="AI...")

In [ ]:
queries = [
    "Explain the theme of the movie?",
    "Who are the characters?",
    "How many male and female characters are in the movie?",
    "Does the script pass the Bechdel test?",
    "What is the role of Deckard in the movie?"
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

retriever = db.as_retriever()

template = """Answer the question based on the following context, If the context does not contain the information to answer the question, return the question that you would like to pass to a search agent to get relevant information to answer the question:
{context}

Question: {question}
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}

)

chain = setup_and_retrieval | prompt | text_llm | output_parser

In [ ]:
for q in queries:
  print(q)
  chain.invoke(q)
  print("-----------------------------------------------------------------------------------------")

In [ ]:
chain.invoke("Explain the theme of the movie?")

'The text provided does not contain any information about the theme of the movie, so I cannot answer this question from the provided context.'

In [ ]:
chain.invoke("Who are the characters?")

'The characters in the context are K, Joshi, Deckard, and Ana.'

In [ ]:
chain.invoke("How many male and female characters are in the movie?")

BlockedPromptException: block_reason: SAFETY
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: LOW
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: MEDIUM
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}


In [ ]:
chain.invoke("Does the script pass the Bechdel test?")

'The provided context does not contain enough information to determine if the script passes the Bechdel test.'

In [ ]:
chain.invoke("What is the role of Deckard in the movie?")

"The provided context does not contain enough information to answer the question about Deckard's role in the movie."